# 1.라이브러리

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import eli5
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')

from tqdm import tqdm
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from eli5.sklearn import PermutationImportance
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

2024-07-09 21:50:32.528544: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# 2.데이터 경로 지정 및 파일 읽기

In [2]:
# data_path = '/home/pervinco/Datasets/UPAI-01-Regression'
data_path = '/Users/pervin0527/upstage-ml-regression-ml11/data'
train_path = f'{data_path}/train.csv'
test_path  = f'{data_path}/test.csv'

print(os.listdir(data_path))

['test.csv', 'subway_feature.csv', 'sampled_data.csv', 'train.csv', 'bus_feature.csv', 'sample_submission.csv']


In [35]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
print('Train data shape : ', train_data.shape, 'Test data shape : ', test_data.shape)

sampled_data = train_data.sample(n=20)
sampled_data.to_csv('sampled_data.csv', index=False)

Train data shape :  (1118822, 52) Test data shape :  (9272, 51)


# 3.컬럼과 결측치 확인

In [4]:
def count_missing_values(df):
    missing_values = df.isna().sum()
    return missing_values

In [5]:
for idx, column in enumerate(train_data.columns, start=1):
    print(f"{idx:>03} : {column}")

001 : 시군구
002 : 번지
003 : 본번
004 : 부번
005 : 아파트명
006 : 전용면적(㎡)
007 : 계약년월
008 : 계약일
009 : 층
010 : 건축년도
011 : 도로명
012 : 해제사유발생일
013 : 등기신청일자
014 : 거래유형
015 : 중개사소재지
016 : k-단지분류(아파트,주상복합등등)
017 : k-전화번호
018 : k-팩스번호
019 : 단지소개기존clob
020 : k-세대타입(분양형태)
021 : k-관리방식
022 : k-복도유형
023 : k-난방방식
024 : k-전체동수
025 : k-전체세대수
026 : k-건설사(시공사)
027 : k-시행사
028 : k-사용검사일-사용승인일
029 : k-연면적
030 : k-주거전용면적
031 : k-관리비부과면적
032 : k-전용면적별세대현황(60㎡이하)
033 : k-전용면적별세대현황(60㎡~85㎡이하)
034 : k-85㎡~135㎡이하
035 : k-135㎡초과
036 : k-홈페이지
037 : k-등록일자
038 : k-수정일자
039 : 고용보험관리번호
040 : 경비비관리형태
041 : 세대전기계약방법
042 : 청소비관리형태
043 : 건축면적
044 : 주차대수
045 : 기타/의무/임대/임의=1/2/3/4
046 : 단지승인일
047 : 사용허가여부
048 : 관리비 업로드
049 : 좌표X
050 : 좌표Y
051 : 단지신청일
052 : target


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118822 entries, 0 to 1118821
Data columns (total 52 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   시군구                     1118822 non-null  object 
 1   번지                      1118597 non-null  object 
 2   본번                      1118747 non-null  float64
 3   부번                      1118747 non-null  float64
 4   아파트명                    1116696 non-null  object 
 5   전용면적(㎡)                 1118822 non-null  float64
 6   계약년월                    1118822 non-null  int64  
 7   계약일                     1118822 non-null  int64  
 8   층                       1118822 non-null  int64  
 9   건축년도                    1118822 non-null  int64  
 10  도로명                     1118822 non-null  object 
 11  해제사유발생일                 5983 non-null     float64
 12  등기신청일자                  1118822 non-null  object 
 13  거래유형                    1118822 non-null  object 
 14  중개

In [7]:
# EDA에 앞서 결측치를 확인해보겠습니다.
train_data.isnull().sum()

시군구                             0
번지                            225
본번                             75
부번                             75
아파트명                         2126
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1112839
등기신청일자                          0
거래유형                            0
중개사소재지                          0
k-단지분류(아파트,주상복합등등)         870691
k-전화번호                     870274
k-팩스번호                     872742
단지소개기존clob                1050240
k-세대타입(분양형태)               869563
k-관리방식                     869563
k-복도유형                     869890
k-난방방식                     869563
k-전체동수                     870630
k-전체세대수                    869563
k-건설사(시공사)                 871058
k-시행사                      871254
k-사용검사일-사용승인일              869696
k-연면적                      869563
k-주거전용면적      

In [8]:
test_data.isnull().sum()

시군구                          0
번지                           2
본번                           0
부번                           0
아파트명                        10
전용면적(㎡)                      0
계약년월                         0
계약일                          0
층                            0
건축년도                         0
도로명                          0
해제사유발생일                   9060
등기신청일자                       0
거래유형                         0
중개사소재지                       0
k-단지분류(아파트,주상복합등등)        6582
k-전화번호                    6576
k-팩스번호                    6606
단지소개기존clob                8718
k-세대타입(분양형태)              6562
k-관리방식                    6562
k-복도유형                    6564
k-난방방식                    6562
k-전체동수                    6577
k-전체세대수                   6562
k-건설사(시공사)                6579
k-시행사                     6580
k-사용검사일-사용승인일             6563
k-연면적                     6562
k-주거전용면적                  6562
k-관리비부과면적                 6562
k-전용면적별세대현황(60㎡이하)        6562
k-전용면적별세

# 3.데이터 전처리

- 데이터에 있는 결측치, 이상치를 처리
- 연속형 변수와 범주형 변수에 대한 확인
- test data에 대해서도 확인이 필요하므로 train, test를 병합하고 진행한다.

In [9]:
# train/test 구분을 위한 칼럼 생성.
train_data['is_test'] = 0
test_data['is_test'] = 1
concat = pd.concat([train_data, test_data]) # 하나의 데이터로 합친다.

In [10]:
concat['is_test'].value_counts() # train과 test data를 하나로 합친 결과.
concat = concat.rename(columns={'전용면적(㎡)':'전용면적'}) # 변수명을 쉽게 변환

In [11]:
# 실제로 결측치라고 표시는 안되어있지만 아무 의미도 갖지 않는 element들이 아래와 같이 존재합니다.
# 아래 3가지의 경우 모두 아무 의미도 갖지 않는 element가 포함되어 있습니다.
display(concat['등기신청일자'].value_counts())

            1111271
20230630        585
20230831        496
20230428        394
20230731        374
             ...   
20230105          3
20230125          3
20230109          1
20230102          1
20230104          1
Name: 등기신청일자, Length: 182, dtype: int64

In [12]:
display(concat['거래유형'].value_counts())

-       1086451
중개거래      38123
직거래        3520
Name: 거래유형, dtype: int64

In [13]:
display(concat['중개사소재지'].value_counts())

-                    1090013
서울 노원구                  2627
서울 송파구                  2540
서울 강남구                  2508
서울 강동구                  2038
                      ...   
서울 강북구, 서울 양천구             1
경기 남양주시, 경기 성남수정구          1
서울 송파구, 인천 계양구             1
경기 연천군, 서울 송파구             1
서울 중랑구, 인천 부평구             1
Name: 중개사소재지, Length: 739, dtype: int64

In [14]:
# 위 처럼 아무 의미도 갖지 않는 칼럼은 결측치와 같은 역할을 하므로, np.nan으로 채워 결측치로 인식되도록 합니다.
concat['등기신청일자'] = concat['등기신청일자'].replace(' ', np.nan)
concat['거래유형'] = concat['거래유형'].replace('-', np.nan)
concat['중개사소재지'] = concat['중개사소재지'].replace('-', np.nan)

In [15]:
# EDA에 앞서 결측치를 확인해보겠습니다.
concat.isnull().sum()

시군구                             0
번지                            227
본번                             75
부번                             75
아파트명                         2136
전용면적                            0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1121899
등기신청일자                    1111271
거래유형                      1086451
중개사소재지                    1090013
k-단지분류(아파트,주상복합등등)         877273
k-전화번호                     876850
k-팩스번호                     879348
단지소개기존clob                1058958
k-세대타입(분양형태)               876125
k-관리방식                     876125
k-복도유형                     876454
k-난방방식                     876125
k-전체동수                     877207
k-전체세대수                    876125
k-건설사(시공사)                 877637
k-시행사                      877834
k-사용검사일-사용승인일              876259
k-연면적                      876125
k-주거전용면적      

In [16]:
concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 9271
Data columns (total 53 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   시군구                     1128094 non-null  object 
 1   번지                      1127867 non-null  object 
 2   본번                      1128019 non-null  float64
 3   부번                      1128019 non-null  float64
 4   아파트명                    1125958 non-null  object 
 5   전용면적                    1128094 non-null  float64
 6   계약년월                    1128094 non-null  int64  
 7   계약일                     1128094 non-null  int64  
 8   층                       1128094 non-null  int64  
 9   건축년도                    1128094 non-null  int64  
 10  도로명                     1128094 non-null  object 
 11  해제사유발생일                 6195 non-null     float64
 12  등기신청일자                  16823 non-null    object 
 13  거래유형                    41643 non-null    object 
 14  중개사소재

In [40]:
def create_corrected_address(row):
    # return f"{row['시군구']}, {row['도로명']}, {row['아파트명']}"
    return f"{row['시군구']}, {row['도로명']}"

address = concat.apply(create_corrected_address, axis=1).to_list()

In [41]:
print(len(address))

address_set = set(address)
print(len(address_set))

1128094
9327


In [42]:
with open('./address.txt', 'w') as file:
    for idx, ad in enumerate(address_set, start=1):
        # names = ad.split(',')
        # names[0] = ' '.join(names[0].split(' ')[:-1])
        
        # address = ''.join(names)
        # file.write(address + '\n')
        # print(idx, address)

        file.write(ad + '\n')
        print(idx, ad)

1 서울특별시 마포구 성산동, 월드컵로26길 13
2 서울특별시 금천구 독산동, 문성로 15
3 서울특별시 구로구 구로동, 도림로3길 3-6
4 서울특별시 양천구 목동, 목동중앙본로1길 5
5 서울특별시 광진구 중곡동, 긴고랑로4길 53
6 서울특별시 도봉구 쌍문동, 시루봉로2길 90
7 서울특별시 양천구 신월동, 화곡로 85
8 서울특별시 중랑구 상봉동, 봉우재로 155
9 서울특별시 구로구 구로동, 디지털로27라길 6-19
10 서울특별시 강서구 화곡동, 등촌로13바길 18-11
11 서울특별시 종로구 신영동, 세검정로6다길 11-1
12 서울특별시 중랑구 묵동, 공릉로2길 8-25
13 서울특별시 동작구 상도동,  
14 서울특별시 강동구 길동, 양재대로122가길 30
15 서울특별시 구로구 구로동, 도림로10길 5-3
16 서울특별시 강서구 화곡동, 월정로 160
17 서울특별시 마포구 창전동, 서강로 95
18 서울특별시 서초구 방배동, 방배로30길 83
19 서울특별시 노원구 공릉동, 공릉로51길 14-17
20 서울특별시 중랑구 면목동, 용마산로40길 27-5
21 서울특별시 강남구 도곡동, 남부순환로 2804
22 서울특별시 강남구 역삼동, 언주로86길 11
23 서울특별시 강남구 청담동, 도산대로83길 36
24 서울특별시 강남구 대치동, 역삼로 436
25 서울특별시 구로구 구로동, 구로동로22길 78-12
26 서울특별시 강동구 강일동, 고덕로 427
27 서울특별시 서초구 신원동, 청계산로9길 1-12
28 서울특별시 동대문구 장안동, 한천로 42
29 서울특별시 성북구 돈암동, 동소문로34길 24
30 서울특별시 동대문구 장안동, 장한로28가길 35
31 서울특별시 강서구 화곡동, 강서로7길 82
32 서울특별시 도봉구 방학동, 시루봉로 71
33 서울특별시 양천구 신월동, 오목로3길 32
34 서울특별시 동대문구 용두동, 청계천로 461
35 서울특별시 강남구 논현동, 언주로 641
36 서울특별시 중랑구 망우동, 봉우재로57길 37-